In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import scrapping_with_selenium as sc  # scrapping_with_selenium: past project which scrapes data from wuzzuf(check it in my past repo)

In [ ]:
# the function which scrapes the data for the firt 150
df = sc.scraper("Data", 150, ["Data Engineer","Analyst", "Data Scientist", "Bi", "Business Intelligence","Data Entry"])

In [ ]:
df.to_csv("D:\Projects\Data Roles Market\df.csv")

In [6]:
# read the csv file 
table = pd.read_csv("D:\Projects\Data Roles Market\df.csv")
table.drop(columns=table.columns[0], axis=1,  inplace=True) # drop first unnamed index column
table.head()

,job title,company,location,experience,salaries,skills,job type,open postions
0,Big Data Management Informatica (BDM) Engineer,Ejada -,"Riyadh, Saudi Arabia",· 3 - 6 Yrs of Exp,Confidential,"['Big Data', 'data quality', 'HBase', 'HDFS', ...",Full Time,2
1,Data Entry,El Sewedy Technical Academy -,"10th of Ramadan City, Sharqia, Egypt",· 1 - 4 Yrs of Exp,5000 To 7000 EGP Per Month,"['Data Entry', 'Data', '#qualityassurance']",Full Time,1
2,Data Scientist,Monty Mobile -,"Cairo, Egypt",· 1 - 10 Yrs of Exp,Confidential,"['Data Science', 'Python', 'Information Techno...",Full Time,2
3,WMS - Data entry coordinator,Logistica -,"6th of October, Giza, Egypt",· 1 - 3 Yrs of Exp,Confidential,"['SAP', 'WMS', 'Data Entry', 'Microsoft Excel']",Full Time,unknown
4,Fleet Data Analyst,Seoudi Supermarket -,"Giza, Egypt",· 3 - 4 Yrs of Exp,Confidential,"['Analyst', 'analytical', 'Data', 'Data Analys...",Full Time,4


In [7]:
table= table[
table["job title"].str.contains("Data") |
table["job title"].str.contains("Analyst") |
table["job title"].str.contains("BI")
]


In [16]:
# create a list of our conditions
conditions = [
table["job title"].str.contains("entry"),
table["job title"].str.contains("Entry"),
table["job title"].str.contains("Analyst"),
table["job title"].str.contains("BI"),
table["job title"].str.contains("Engineer"),
table["job title"].str.contains("Scientist")
]

# create a list of the values we want to assign for each condition
values = ['data entry', 'data entry', 'data analyst', 'bi developer','data engineer', 'data scientist']

# create a new column and use np.select to assign values to it using our lists as arguments
table['roles'] = np.select(conditions, values)

# display updated DataFrame
table.head()

,job title,company,location,experience,salaries,skills,job type,open postions,roles
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
128,False,False,False,False,False,False,False,False,False
129,False,False,False,False,False,False,False,False,False
130,False,False,False,False,False,False,False,False,False
131,False,False,False,False,False,False,False,False,False
